# 01 — Surprise Metric: Đo "Độ Bất Ngờ" Của Dữ Liệu

Trong kiến trúc **Titans**, cơ chế **Surprise Metric** quyết định thông tin nào đáng ghi nhớ.

**Ý tưởng cốt lõi:**
- Một mạng neural nhỏ (predictor) học dự đoán dữ liệu đầu vào
- **Surprise = sai số dự đoán** (MSE giữa dự đoán và thực tế)
- Dữ liệu quen thuộc → surprise thấp → bỏ qua
- Dữ liệu mới lạ → surprise cao → ghi nhớ!

Giống cách bộ não con người: bạn không nhớ mọi bước chân, nhưng sẽ nhớ rõ khi vấp ngã.

In [ ]:
import torch
import matplotlib.pyplot as plt
from titans_memory import SurpriseMetric, generate_repeating_with_anomalies

# Tạo chuỗi số lặp lại [1,2,3,1,2,3,...] với vài điểm bất thường
seq, anomaly_mask = generate_repeating_with_anomalies(
    pattern=[1.0, 2.0, 3.0],
    repeats=20,
    anomaly_indices=[14, 35, 50],
    anomaly_value=99.0,
)

print(f"Độ dài chuỗi: {len(seq)}")
print(f"Số anomaly: {anomaly_mask.sum().int().item()}")
print(f"Vị trí anomaly: {anomaly_mask.nonzero().squeeze(-1).tolist()}")
print(f"Mẫu đầu: {seq[:12].tolist()}")

## Cách Surprise Metric hoạt động

`SurpriseMetric` chứa một **predictor network** (MLP nhỏ) cố gắng dự đoán giá trị tiếp theo.

1. Mỗi bước, predictor đưa ra dự đoán
2. **Surprise = MSE(dự đoán, thực tế)**
3. Predictor được cập nhật để học pattern → surprise giảm dần cho dữ liệu quen
4. Khi gặp anomaly → predictor dự đoán sai → surprise tăng vọt!

In [ ]:
torch.manual_seed(42)
metric = SurpriseMetric(input_dim=1, hidden_dim=16)

surprise_scores = []
for i in range(len(seq)):
    x = seq[i].unsqueeze(0)  # shape: (1,)
    score = metric(x)
    surprise_scores.append(score.item())
    metric.update_predictor(x, lr=0.01)

print(f"Surprise trung bình: {sum(surprise_scores)/len(surprise_scores):.4f}")
print(f"Surprise tại anomaly[0] (idx=14): {surprise_scores[14]:.4f}")
print(f"Surprise tại vị trí bình thường (idx=13): {surprise_scores[13]:.4f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Biểu đồ trên: chuỗi input
ax1.plot(seq.numpy(), color="#2196F3", linewidth=1, label="Input sequence")
anomaly_idx = anomaly_mask.nonzero().squeeze(-1).numpy()
ax1.scatter(anomaly_idx, seq[anomaly_mask.bool()].numpy(),
            color="#F44336", s=80, zorder=5, label="Anomalies")
ax1.set_ylabel("Giá trị")
ax1.set_title("Chuỗi Input: Pattern lặp lại [1,2,3] với các điểm bất thường")
ax1.legend()

# Biểu đồ dưới: surprise scores
ax2.plot(surprise_scores, color="#FF9800", linewidth=1.5, label="Surprise score")
for idx in anomaly_idx:
    ax2.axvline(x=idx, color="#F44336", alpha=0.3, linestyle="--")
ax2.set_xlabel("Timestep")
ax2.set_ylabel("Surprise Score")
ax2.set_title("Surprise Score: Tăng vọt tại các điểm bất thường!")
ax2.legend()

plt.tight_layout()
plt.show()

## Kết luận

- Surprise score **giảm dần** khi predictor học được pattern lặp lại [1, 2, 3]
- Tại các điểm anomaly (giá trị = 99), surprise **tăng vọt** vì predictor không thể dự đoán được
- Sau anomaly, surprise quay về mức thấp vì pattern quen thuộc tiếp tục

→ Đây chính là cơ sở để Titans **chỉ ghi nhớ những gì bất ngờ**, tiết kiệm bộ nhớ dài hạn.